# PermRowCol experimental results

In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 

from multiprocessing import Pool

from pyzx import cnot_mapper, architecture # Note that this is the local pyzx package from this repository/fork, not the one installed through pip or other means.

ImportError: cannot import name 'cnot_mapper' from 'pyzx' (/home/ewan/.local/lib/python3.10/site-packages/pyzx/__init__.py)

First, we specify some code to read the dataset of CNOT circuits that was used in [ArXiv 1904.00633](https://arxiv.org/pdf/1904.00633.pdf). Since the original paper only used 20 circuits per test case (number of qubits, number of original CNOTs), we generated 80 more circuits that are now available on this GitHub repository in the folder circuits/steiner/

In [ ]:
def read_circuit(source):
    if not os.path.exists(source):
        print("File {} does not exist".format(source))
        return
    return cnot_mapper.CNOT_tracker.from_qasm_file(source, parities_as_columns=True)

def read_circuits(n_qubits):
    source_folder = "../circuits/steiner/"
    circuits = []
    sources = []
    subfolder = os.path.join(source_folder, str(n_qubits)+"qubits/")

    for folder in os.listdir(subfolder):
        folder = os.path.join(subfolder, folder)
        if os.path.isdir(folder):
            for file in os.listdir(folder):
                if file.endswith(".qasm"):
                    src = os.path.join(folder, file)
                    circuit = read_circuit(src)
                    circuits.append(circuit)
                    sources.append(src)
    return circuits, sources

Then we run our experiment with the algorithms as they are.

In [ ]:
def multiprocess_experiments(mode, circuits, arch):
    pool = Pool(processes=os.cpu_count())
    n_circuits = len(circuits)
    inputs = zip([mode]*n_circuits, 
                 [cnot_mapper.Mat2(np.copy(c.matrix.data)) for c in circuits], 
                 [arch]*n_circuits,
                 [True]*n_circuits
                )
    new_circuits =  pool.starmap(cnot_mapper.gauss_return_circuit, inputs)
    return [c.count_cnots() for c in new_circuits]

def run_steiner_gauss(circuits, arch):
    return multiprocess_experiments(cnot_mapper.STEINER_MODE, circuits, arch)

def run_rowcol(circuits, arch):
    return multiprocess_experiments(cnot_mapper.ROWCOL_MODE, circuits, arch)

def run_perm_rowcol(circuits, arch):
    return multiprocess_experiments(cnot_mapper.PERMROWCOL_MODE, circuits, arch)

# Reverse traversal parameters
RT_parameters = {
    "max_iter" : 15
}

Astar_parameters = {
    "choiceWidth": 4,
    "parities_as_columns": True,
    "max_size": 10
}

def run_reverse_traversal(circuits, arch): 
    pool = Pool(os.cpu_count())
    n_circuits = len(circuits)
    inputs = zip([cnot_mapper.Mat2(np.copy(c.matrix.data)) for c in circuits], [arch]*n_circuits, [RT_parameters["max_iter"]]*n_circuits)
    new_circuits = pool.starmap(cnot_mapper.reverse_traversal, inputs)
    return [c.count_cnots() for c,i,o in new_circuits]

def run_A_permrowcol(circuits, arch):
    pool = Pool(os.cpu_count())
    n_circuits = len(circuits)
    inputs = zip([cnot_mapper.Mat2(np.copy(c.matrix.data)) for c in circuits], [arch]*n_circuits, 
                 [Astar_parameters["choiceWidth"]]*n_circuits,
                [Astar_parameters["max_size"]]*n_circuits,
                [Astar_parameters["parities_as_columns"]]*n_circuits)
    new_circuits = pool.starmap(cnot_mapper.A_permrowcol, inputs)
    return [c.count_cnots() for p,c in new_circuits]


def run_Astart_reverse_traversal(circuits, arch): 
    pool = Pool(os.cpu_count())    
    n_circuits = len(circuits)
    inputs = zip([cnot_mapper.Mat2(np.copy(c.matrix.data)) for c in circuits], [arch]*n_circuits, [RT_parameters["max_iter"]]*n_circuits,  [Astar_parameters]*n_circuits)
    new_circuits = pool.starmap(cnot_mapper.reverse_traversal, inputs)
    return [c.count_cnots() for c,i,o in new_circuits]

In [ ]:
def run_experiment(arch, method):
    n_qubits = arch.n_qubits
    og_circuits, srcs = read_circuits(n_qubits)
    results = pd.DataFrame()

    results["Original"] = np.array([int(src.split("/")[-2]) for src in srcs])
    results["#Qubits"] = np.array([n_qubits]*len(og_circuits))
    results["Architecture"] = np.array([arch.name]*len(og_circuits))
    
    for m, func in method.items():
        print("Running", m)
        results[(m, "count")] = np.array(func(og_circuits, arch))
        results[(m, "overhead (%)")] = (results[(m, "count")]/results["Original"] - 1)*100
        tmp = results.groupby(["Original", "Architecture", "#Qubits"]).mean()
        tmp.columns = pd.MultiIndex.from_tuples(tmp.columns.tolist()) 
        tmp.to_csv("tmp/"+m+arch.name+" results.csv")

    results = results.groupby(["Original", "Architecture", "#Qubits"]).mean()

    results.columns = pd.MultiIndex.from_tuples(results.columns.tolist()) 
    results.to_csv("tmp/"+arch.name+" results.csv")
    return results

In [ ]:
archs = [
    architecture.create_architecture(architecture.SQUARE, n_qubits=9),
    architecture.create_architecture(architecture.SQUARE, n_qubits=16),
    architecture.create_architecture(architecture.RIGETTI_16Q_ASPEN),
    architecture.create_architecture(architecture.IBM_QX5),
    architecture.create_architecture(architecture.IBM_Q20_TOKYO)
]

method = {
    "SteinerGauss": run_steiner_gauss,
    "RowCol": run_rowcol,
    "PermRowCol without RT": run_perm_rowcol,
    "PermRowCol with RT": run_reverse_traversal
}

results = pd.concat([run_experiment(arch, method) for arch in archs])
results.to_csv("PermRowCol results.csv")

In [ ]:
results

In [ ]:
archs = [
    architecture.create_architecture(architecture.FULLY_CONNNECTED, n_qubits=5),
    architecture.create_architecture(architecture.LINE, n_qubits=5),
    architecture.create_architecture(architecture.IBM_QX2),
    architecture.create_architecture(architecture.IBM_QX4)
]
method = {
    "SteinerGauss": run_steiner_gauss,
    "RowCol": run_rowcol,
    "PermRowCol without RT": run_perm_rowcol,
    "PermRowCol with RT": run_reverse_traversal,
    "PermRowCol with A*": run_A_permrowcol,
    "PermRowCol with RT and A*": run_Astart_reverse_traversal
}

results = pd.concat([run_experiment(arch, method) for arch in archs])
results.to_csv("A* PermRowCol results.csv")

In [ ]:
results

The PermRowCol algorithm relies on heuristical functions for determining which row and column to pick. Below are some example functions to to play around with. By default, the behavior uses `ChooseRowMin` and `ChooseRowMax` (hardcoded in `permrowcol`).

In [ ]:
def chooseRowMin(m, o):
    return o[np.argmin([sum(m.data[i]) for i in o])]

def chooseColumnMin(m, r, o):
    return o[np.argmin([sum(m.data[:, i]) if m.data[r][i] == 1 else len(m.data) for i in o])] 

def chooseRowMax(m, o):
    return o[np.argmax([sum(m.data[i]) for i in o])]

def chooseColumnMax(m, r, o):
    return o[np.argmax([sum(m.data[:, i]) if m.data[r][i] == 1 else 0 for i in o])] 

## Unconstrained experiments
Because there are more applications for Gaussian elimination where the number of steps are counted and the resulting matrix can also be a permutation matrix, we also compare our algorithm in case that there are no connectivity constraints. 

For example, this can be useful in ZX diagram extraction.

In [ ]:
fully_connected_archs = [architecture.create_architecture(architecture.FULLY_CONNNECTED, n_qubits=n) for n in [5, 9, 16, 20]]

In [ ]:
method = {
    "SteinerGauss": run_steiner_gauss,
    "RowCol": run_rowcol,
    "PermRowCol without RT": run_perm_rowcol,
    "PermRowCol with RT": run_reverse_traversal
}
results = pd.concat([run_experiment(arch, method) for arch in fully_connected_archs])
results.to_csv("PermRowCol results UNCONSTRAINED.csv")

In [ ]:
results

# Visualization


In [ ]:
def plot_results(filename, architectures, n=4):
    df = pd.read_csv(filename, header=[0,1,2])
    for i, arch in enumerate(architectures):
        tmp_df = df[df[df.columns[1]] == arch.name]
        
        original = tmp_df[df.columns[0]]
        steinergauss = tmp_df[df.columns[3]]
        rowcol = tmp_df[df.columns[5]]
        permrowcol = tmp_df[df.columns[7]]

        plt.figure(i)
        plt.plot(original, original, label="x=y")
        for i in range(n):
            method = df.columns[2*i+3]
            data = tmp_df[method]
            plt.plot(original, data, label=method[0])
        plt.title(arch.name)
        plt.xlabel("Original CNOT count")
        plt.ylabel("Routed CNOT count")
        plt.legend()
        plt.savefig("figs/"+filename[:-4]+arch.name, dpi=150)
        plt.show()


In [ ]:
filename = "PermRowCol results.csv"
archs = [
    architecture.create_architecture(architecture.SQUARE, n_qubits=9),
    architecture.create_architecture(architecture.SQUARE, n_qubits=16),
    architecture.create_architecture(architecture.RIGETTI_16Q_ASPEN),
    architecture.create_architecture(architecture.IBM_QX5),
    architecture.create_architecture(architecture.IBM_Q20_TOKYO)
]
plot_results(filename, archs)

In [ ]:
filename = "A* PermRowCol results.csv"
archs = [
    architecture.create_architecture(architecture.FULLY_CONNNECTED, n_qubits=5),
    architecture.create_architecture(architecture.LINE, n_qubits=5),
    architecture.create_architecture(architecture.IBM_QX2),
    architecture.create_architecture(architecture.IBM_QX4)
]
plot_results(filename, archs, n=6)

In [ ]:
filename = "PermRowCol results UNCONSTRAINED.csv"
fully_connected_archs = [architecture.create_architecture(architecture.FULLY_CONNNECTED, n_qubits=n) for n in [5, 9, 16, 20]]
plot_results(filename, fully_connected_archs)